In [171]:
import json
import sqlite3
import re

In [165]:
FILEPATH = 'C:/Users/User/Desktop/Python/PythonProjects/Twitter_info_project/three_minutes_tweets.json.txt'
FILEPATH_AF = 'C:/Users/User/Desktop/Python/PythonProjects/Twitter_info_project/AFINN-111.txt'
db = 'mts_twits.db'

In [175]:
class Sqlite_Contextman:
    
    def __init__(self, dbname):
        self.dbname = dbname
        
    def __enter__(self):
        self.conn = sqlite3.connect(self.dbname)
        return self.conn
    
    def __exit__(self, exc_val, exc_type, exc_tb):
        self.conn.close()
        if exc_val:
            raise
    
    @staticmethod
    def db_querrier(cursor, sql):
        """Для запуска произвольного запроса"""
        cursor.execute(sql)
        
def tweets_inserter_querry_maker(cursor, line):
    """Функция для записи json в базу данных"""
    country_code = None
    if line['place'] != None:
        country_code = line['place']['country_code']
    cursor.execute("""insert or ignore into mytab (id, created_at, tweet_text, name, location, country_code, lang) 
            values (?,?,?,?,?,?,?)""",[line['id'],
                                       line['created_at'],
                                       line['text'],
                                       line['user']['name'],
                                       line['user']['location'],
                                       country_code,
                                       line['user']['lang']])

def tweets_inserter(db, FILEPATH):
    counter = 0
    with Sqlite_Contextman(db) as conn:
        cursor = conn.cursor()
        line = True
        with open(FILEPATH) as filetoread:
            try:
                while line:
                    line = json.loads(filetoread.readline())
                    if 'delete' not in line.keys():
                        counter += 1
                        tweets_inserter_querry_maker(cursor, line)
                        if counter % 1000 == 0: print(counter)
                    
            except Exception as err:
                print(counter, line['text'], err)
                temp = line
                conn.commit()
        conn.commit()

def sentiment_inserter(db, FILEPATH):
    line = True
    with Sqlite_Contextman(db) as conn:
        cursor = conn.cursor()
        with open(FILEPATH_AF) as filetoread:
            try:
                while line != '':
                    line = filetoread.readline()
                    sentiment = re.split('\t', line)
                    sentiment[1] = int(sentiment[1])
                    cursor.execute("""insert or ignore into tweet_sentiment (word, val) values (?,?)""",(sentiment[0], sentiment[1]))
            except Exception as err:
                print(err, line == '')
                conn.commit()
        conn.commit()

In [177]:
tweets_inserter(db, FILEPATH)
sentiment_inserter(db, FILEPATH_AF)

1000
2000
3000
4000
5000
6000
7000
7338 @TetoKasane11 yo igual soy un neko hermana,nya http://t.co/O323oIMbgI Expecting value: line 1 column 1 (char 0)
list index out of range True


In [155]:
# sql = """create table if not exists tweet_sentiment (word text not null primary key, val int)"""
# # sql = """drop table tweet_sentiment"""
# # sql = """insert or ignore into tweet_sentiment (word, val) values ()"""
# with Sqlite_Contextman(db) as conn:
#     cursor = conn.cursor()
#     Sqlite_Contextman.db_querrier(cursor, sql)
#     conn.commit()

1. Нужно написать генератор словаря или списка для sentiment
2. Нужно "на лету" обсчитывать sentiment

In [ ]:
with open(FILEPATH_AF) as filetoread:
        try:
            while line != '':
                line = filetoread.readline()
                sentiment = re.split('\t', line)
                sentiment[1] = int(sentiment[1])
                
        except Exception as err:
            print(err, line == '')